<a href="https://colab.research.google.com/github/SDS-AAU/SDS-master/blob/master/M1/notebooks/dsba_workshop2_Predicting_AirBnb_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
listings = pd.read_csv('http://data.insideairbnb.com/denmark/hovedstaden/copenhagen/2020-06-26/data/listings.csv.gz', compression='gzip')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (43,61,62,94) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
listings.price = listings.price.str.replace(',','')
listings.price = listings.price.str.replace('$','')
listings.price = listings.price.astype(float)

In [ ]:
listings.cleaning_fee = listings.cleaning_fee.str.replace(',','')
listings.cleaning_fee = listings.cleaning_fee.str.replace('$','')
listings.cleaning_fee = listings.cleaning_fee.astype(float)

In [ ]:
listings['price_total'] = listings.price + listings.cleaning_fee

In [ ]:
listings_clean = listings[listings.price_total < listings.price_total.quantile(0.85)]

In [ ]:
listings_clean = listings_clean[listings.number_of_reviews > 10]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [ ]:
listings_clean.head(1).loc[:,'neighbourhood_cleansed':]

,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,price_total
0,Nrrebro,NaN,Copenhagen,Hovedstaden,2200,Copenhagen,"Copenhagen, Denmark",DK,Denmark,55.68798,12.54571,t,Apartment,Private room,2,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Wifi,Kitchen,""Paid parking off ...",97.0,365.0,NaN,NaN,$0.00,33.0,1,$66.00,2,15,2,2,15,15,2.0,15.0,5 months ago,t,29,59,89,89,2020-06-28,168,1,2009-09-04,2019-07-19,96.0,10.0,9.0,10.0,10.0,9.0,9.0,f,NaN,NaN,f,f,moderate,f,f,1,0,1,0,1.28,398.0


In [ ]:
listings_clean.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6578 entries, 0 to 27378
Data columns (total 107 columns):
 #   Column                                        Dtype  
---  ------                                        -----  
 0   id                                            int64  
 1   listing_url                                   object 
 2   scrape_id                                     int64  
 3   last_scraped                                  object 
 4   name                                          object 
 5   summary                                       object 
 6   space                                         object 
 7   description                                   object 
 8   experiences_offered                           object 
 9   neighborhood_overview                         object 
 10  notes                                         object 
 11  transit                                       object 
 12  access                                        object 
 13  i

In [ ]:
listings_clean['property_type'].value_counts()

Apartment             5554
Condominium            563
House                  185
Townhouse              100
Loft                    82
Villa                   21
Guest suite             14
Guesthouse              13
Hostel                   9
Bungalow                 8
Houseboat                7
Boat                     6
Bed and breakfast        4
Serviced apartment       4
Tiny house               3
Cottage                  1
Boutique hotel           1
Cabin                    1
Hotel                    1
Hut                      1
Name: property_type, dtype: int64

In [ ]:
listings_clean['bed_type'].value_counts()

Real Bed         6428
Pull-out Sofa      88
Futon              40
Couch              12
Airbed             10
Name: bed_type, dtype: int64

In [ ]:
listings_clean = listings_clean[listings_clean.property_type.isin(['Apartment','Condominium','House'])]
listings_clean = listings_clean[listings_clean.bed_type.isin(['Real Bed','Pull-out Sofa'])]

In [ ]:
y = listings_clean.price_total

In [ ]:
X = listings_clean.iloc[:,[39,51,52,53,54,55,56,57]]

In [ ]:
X

,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type
0,Nrrebro,Apartment,Private room,2,1.0,1.0,1.0,Real Bed
3,Vesterbro-Kongens Enghave,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed
4,sterbro,Apartment,Entire home/apt,4,1.0,3.0,3.0,Real Bed
6,Vesterbro-Kongens Enghave,Apartment,Entire home/apt,3,2.0,1.0,2.0,Real Bed
8,Vesterbro-Kongens Enghave,Apartment,Entire home/apt,5,2.0,2.0,1.0,Real Bed
...,...,...,...,...,...,...,...,...
26850,Amager Vest,Apartment,Private room,4,0.5,1.0,4.0,Real Bed
26870,sterbro,Apartment,Entire home/apt,5,1.5,4.0,4.0,Real Bed
26962,sterbro,Apartment,Entire home/apt,4,1.0,2.0,3.0,Real Bed
27181,Amager st,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed


In [ ]:
X = pd.get_dummies(X)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6243 entries, 0 to 27349
Data columns (total 23 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   accommodates                                      6243 non-null   int64  
 1   bathrooms                                         6243 non-null   float64
 2   bedrooms                                          6241 non-null   float64
 3   beds                                              6242 non-null   float64
 4   neighbourhood_cleansed_Amager Vest                6243 non-null   uint8  
 5   neighbourhood_cleansed_Amager st                  6243 non-null   uint8  
 6   neighbourhood_cleansed_Bispebjerg                 6243 non-null   uint8  
 7   neighbourhood_cleansed_Brnshj-Husum               6243 non-null   uint8  
 8   neighbourhood_cleansed_Frederiksberg              6243 non-null   uint8  
 9   neighbourhood_clea

In [ ]:
X.iloc[:,:4] = scaler.fit_transform(X.iloc[:,:4])

In [ ]:
from fancyimpute import SoftImpute

In [ ]:
X.iloc[:,:4] = SoftImpute().fit_transform(X.iloc[:,:4])

[SoftImpute] Max Singular Value of X_init = 124.757562
[SoftImpute] Iter 1: observed MAE=0.020944 rank=4
[SoftImpute] Iter 2: observed MAE=0.020944 rank=4
[SoftImpute] Iter 3: observed MAE=0.020944 rank=4
[SoftImpute] Iter 4: observed MAE=0.020944 rank=4
[SoftImpute] Iter 5: observed MAE=0.020944 rank=4
[SoftImpute] Iter 6: observed MAE=0.020944 rank=4
[SoftImpute] Iter 7: observed MAE=0.020944 rank=4
[SoftImpute] Iter 8: observed MAE=0.020944 rank=4
[SoftImpute] Iter 9: observed MAE=0.020944 rank=4
[SoftImpute] Iter 10: observed MAE=0.020944 rank=4
[SoftImpute] Iter 11: observed MAE=0.020944 rank=4
[SoftImpute] Iter 12: observed MAE=0.020944 rank=4
[SoftImpute] Iter 13: observed MAE=0.020944 rank=4
[SoftImpute] Iter 14: observed MAE=0.020944 rank=4
[SoftImpute] Iter 15: observed MAE=0.020944 rank=4
[SoftImpute] Iter 16: observed MAE=0.020944 rank=4
[SoftImpute] Iter 17: observed MAE=0.020944 rank=4
[SoftImpute] Iter 18: observed MAE=0.020944 rank=4
[SoftImpute] Iter 19: observed MAE=0

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 21)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model_ols = LinearRegression()

In [ ]:
model_ols.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
model_ols.score(X_test, y_test)

0.5439291006415563

In [ ]:
from xgboost import XGBRegressor

In [ ]:
model_xgb = XGBRegressor()

In [ ]:
model_xgb.fit(X_train, y_train)

[18:02:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
model_xgb.score(X_test, y_test)

0.5518604964744585

In [ ]:
import numpy as np

In [ ]:
np.sqrt(mean_squared_error(y_test, model_xgb.predict(X_test)))

231.47256340394404

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer 
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score

In [ ]:
scorer = make_scorer(r2_score)

In [ ]:
parameters_xgb = { 'max_depth': [3, 5, 7],
 'n_estimators': [25, 50, 100]}

In [ ]:
# erform grid search on the classifier using 'scorer' as the scoring method.
grid_obj = GridSearchCV(model_xgb, parameters_xgb, scoring=scorer)

In [ ]:
grid_fit = grid_obj.fit(X_train, y_train)

[17:24:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:24:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:24:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:24:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:24:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:24:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:24:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:24:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [ ]:
# Get the estimator.
best_reg = grid_fit.best_estimator_

# Fit the new model.
best_reg.fit(X_train, y_train)

[17:26:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
best_reg.score(X_test, y_test)

0.45544695445489913